In [6]:
from qibo.models.variational import VQE
from boostvqe.ansatze import build_circuit
from qibo import hamiltonians
from pathlib import Path

In [7]:
# insert path to parameters_history.npy
path = Path("../results/pure_vqe/BFGS_8q_1l_42/parameters_history.npy")

In [8]:
import numpy as np

In [9]:
params = np.load(path, allow_pickle=True).tolist()

In [10]:
params[0][-1]

array([ 1.01826928,  3.42201683,  0.11638393,  1.55161516, -2.95835472,
       -2.74524029, -3.59575963,  0.63840512, -0.97780105,  1.8301649 ,
       -3.33491212,  3.57706204,  1.74313567, -1.60208323, -2.49052699,
       -1.91391829, -1.7834659 ,  1.51016373,  0.75563978, -2.29154434,
        1.47091912, -2.46513894, -1.55926372,  0.54112274,  1.01884751,
        2.99077176, -3.16998465,  0.63962461,  0.1747779 , -2.5445537 ,
        0.24075282, -0.72043619, -2.31949996,  3.09109558,  1.54556804,
        1.53117995, -0.04777816, -1.84636061, -0.38710555, -0.95312714])

In [11]:
circuit = build_circuit(nqubits=8, nlayers=1)
ham = hamiltonians.XXZ(nqubits=8)

[Qibo 0.2.8|INFO|2024-05-30 19:23:23]: Using qibojit (numba) backend on /CPU:0


In [12]:
vqe = VQE(circuit=circuit, hamiltonian=ham)

In [13]:
vqe.circuit.set_parameters(params[0][-1])

In [14]:
final_state = vqe.circuit().state()

final_state

In [16]:
ham.expectation(final_state)

-10.466986134259724

In [17]:
from qibo import hamiltonians, set_backend
from qibo.models.dbi.double_bracket import DoubleBracketGeneratorType, DoubleBracketScheduling, DoubleBracketIteration, DoubleBracketCostFunction
from qibo.models.dbi.utils import *
from qibo.models.dbi.utils_scheduling import *

In [21]:
u =  vqe.circuit.unitary()
h_vqe = u.conj().T @ ham.matrix @ u
from numpy.linalg import norm
from boostvqe.utils import rotate_h_with_vqe
norm(h_vqe - rotate_h_with_vqe(ham, vqe) )

0.0

In [53]:
cost = DoubleBracketCostFunction.least_squares
dbi = DoubleBracketIteration(hamiltonians.Hamiltonian(nqubits= nqubits, matrix=h_vqe),
                             mode=DoubleBracketGeneratorType.group_commutator,cost=cost)

In [54]:
dbi_eval = deepcopy(dbi)
flows = 1

d_opt_symbolic = SymbolicHamiltonian( symbols.Z(1)+symbols.Z(2)+symbols.Z(3)+symbols.Z(4)+symbols.Z(5)+symbols.Z(6),nqubits=8)


v = dbi_eval.eval_dbr_unitary(0.02,d=d_opt_symbolic.dense.matrix)

ket_ini = np.transpose(dbi.h.backend.zero_state(nqubits))
dbi.h.expectation(v.conj().T@ket_ini)

[Qibo 0.2.8|WARNING|2024-05-30 19:52:52]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


-10.90216959139352

In [56]:
ham.expectation(u@v.conj().T@ket_ini)

-10.90216959139353